In [ ]:
import covid
import numpy as np
from datetime import datetime,timedelta, date
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator, FuncFormatter
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from IPython.display import clear_output
from IPython.display import Markdown
from cycler import cycler
import ipywidgets as widgets
%matplotlib inline

In [ ]:
# Config

days_range = (date.today() - date(2020,2,24)).days + 5
order = 8
log_scale = False
tick = FuncFormatter(lambda x, p: format(int(x), ','))

covid.load_regional()
data = covid.get_region(covid.data_reg, 'Lombardia')
updated_at = covid.updated_at

# Implementation

def fmt_plot(aplot, size):
  global log_scale

  aplot.grid(which='major', color='#999999', linestyle='--')
  aplot.grid(which='minor', color='#CCCCCC', linestyle=':')
  aplot.set(xlabel="Data")
  aplot.yaxis.set_major_formatter(tick)
  if log_scale:
    return aplot
  aplot.axvline(x=datetime(2020,3,9), color="orange", linestyle="--")
  aplot.axvline(x=datetime(2020,3,21), color="red", linestyle="--")

  return aplot

def format_row_wise(styler, formatter):
    for row, row_formatter in formatter.items():
        row_num = styler.index.get_loc(row)
        for col_num in range(len(styler.columns)):
            styler._display_funcs[(row_num, col_num)] = row_formatter            
    return styler

def draw_reflines(plt, label=False):
    if (label==True):
        plt.axvline(x=datetime(2020,3,9), color="orange", label="lockdown iniziale", linestyle="--")
        plt.axvline(x=datetime(2020,3,21), color="red", label="lockdown completo", linestyle="--")
        plt.axvline(x=datetime(2020,5,3), color="lightgreen", label="riapertura iniziale", linestyle="--")
        plt.axvline(x=datetime(2020,5,17), color="green", label="riapertura completa", linestyle="--")
        plt.axvline(x=datetime(2020,6,2), color="blue", label="spostamenti tra regioni", linestyle="--")
    else:
        plt.axvline(x=datetime(2020,3,9), color="orange", linestyle="--")
        plt.axvline(x=datetime(2020,3,21), color="red", linestyle="--")
        plt.axvline(x=datetime(2020,5,3), color="lightgreen", linestyle="--")
        plt.axvline(x=datetime(2020,5,17), color="green", linestyle="--")    
        plt.axvline(x=datetime(2020,6,2), color="blue", linestyle="--")

def color_negative(val):    
    color = 'red' if val < 0 else 'black'
    weight = 'bold' if val < 0 else 'normal'
    return 'color: %s; font-weight:%s' % (color, weight)

styles = [
    dict(selector="*", props=[("font-size", "105%")])
]


# Dati Coronavirus Piemonte

In [ ]:
# Data model:
#   'data', 'stato', 'ricoverati_con_sintomi', 'terapia_intensiva',
#   'totale_ospedalizzati', 'isolamento_domiciliare', 'totale_positivi',
#   'variazione_totale_positivi', 'nuovi_positivi', 'dimessi_guariti',
#   'deceduti', 'totale_casi', 'tamponi', 'note_it', 'note_en'


display(Markdown("## *Aggiornamento al " + updated_at + "*"))



## Grafici andamenti 


In [ ]:
plt.rcParams['figure.figsize'] = [16, 5]
plt.rcParams['axes.prop_cycle'] = cycler(color='byrgmgk')

#create tabs
out1 = widgets.Output()
out2 = widgets.Output()
tab_nest = widgets.Tab(children = [out1, out2])
tab_nest.set_title(0, 'Lineari')
tab_nest.set_title(1, 'Logaritmici')

def smooth(datas):
    return datas.rolling(7,
        win_type='gaussian',
        min_periods=1,
        center=True).mean(std=2).round()


def plot_func(columns):    
    global log_scale
    global data
    return data.loc[:,columns] \
            .plot(kind='line',
                subplots=True,
                sharex=True,
                layout=(1,3),
                grid=True,
                legend=True,
                logy=log_scale)

def update_legend(plot, manual=None):
    handles, labels = plot.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plot.legend(by_label.values(), by_label.keys())

def draw_plots(): 
    avg_days = 7    
    plt.rcParams['figure.figsize'] = [16, 5]    
    
    plots = plot_func(['totale_positivi','variazione_totale_positivi','terapia_intensiva'])
    
    plt_casi = fmt_plot(plots[0][0], 20000)
    draw_reflines(plt_casi, True)
    moving_avg = smooth(data.variazione_totale_positivi)
    update_legend(plt_casi)
    
    plt_positivi = fmt_plot(plots[0][1], 1000)    
    plt_positivi.plot(moving_avg, color='green', linestyle='-.', label='media mobile')
    draw_reflines(plt_positivi)
    update_legend(plt_positivi)
        
    plt_intensivi = fmt_plot(plots[0][2], 1000)
    draw_reflines(plt_intensivi)    
    update_legend(plt_intensivi)
    
    plt.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')
    plots = plot_func(['totale_ospedalizzati','deceduti','dimessi_guariti'])
    plt_ricoverati = fmt_plot(plots[0][0], 10000)
    draw_reflines(plt_ricoverati)
    update_legend(plt_ricoverati)

    plt_deceduti = fmt_plot(plots[0][1], 5000)
    draw_reflines(plt_deceduti)
    update_legend(plt_deceduti)
    
    plt_dimessi = fmt_plot(plots[0][2], 20000)
    draw_reflines(plt_dimessi)
    update_legend(plt_dimessi)
    
    plt.rcParams['axes.prop_cycle'] = cycler(color='yyrrmgk')
    plots = plot_func(['nuovi_decessi','nuovi_positivi','tamponi'])

    plt_nuovi_decessi = fmt_plot(plots[0][0], 200)
    moving_avg = smooth(data.nuovi_decessi)
    plt_nuovi_decessi.plot(moving_avg, color='blue', label='media mobile', linestyle='-.')
    draw_reflines(plt_nuovi_decessi)    
    update_legend(plt_nuovi_decessi)

    
    plt_nuovi = fmt_plot(plots[0][1], 1000)
    moving_avg = smooth(data.nuovi_positivi)
    plt_nuovi.plot(moving_avg, color='green', label='media mobile', linestyle='-.')
    draw_reflines(plt_nuovi)
    update_legend(plt_nuovi)
    
    plt_tamponi = fmt_plot(plots[0][2], 500000)
    draw_reflines(plt_tamponi)
    update_legend(plt_tamponi)
                      
    plt.show()

log_scale = False    
with out1:        
    draw_plots()
log_scale = True
with out2:
    plt.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')
    draw_plots()
    
display(tab_nest)


## Andamento casi

In [ ]:
tick = FuncFormatter(lambda x, p: format(int(x), ','))
plt.rcParams['figure.figsize'] = [16, 10]

def annot_max(x,y, ax=None):
    xmax = x[np.argmax(y)]
    ymax = y.max()
    text= "data={:%d/%m/%Y}, max={:,.0f}".format(xmax, ymax)
    if not ax:
        ax=plt.gca()
    bbox_props = dict(boxstyle="square,pad=0.5", fc="w", ec="k", lw=0.52)
    arrowprops=dict(arrowstyle="->",connectionstyle="angle,angleA=0,angleB=60")
    kw = dict(xycoords='data',textcoords="axes fraction",fontsize=12,
              arrowprops=arrowprops, bbox=bbox_props, ha="right", va="top")
    ax.annotate(text, xy=(xmax, ymax), xytext=(0.64,0.96), **kw)
    
def draw_scatters():
    aplot = plt.gca()
    fmt_plot(aplot, 150000)
    plt.ylabel("Numero totale")
    plt.xticks(rotation=30)
    aplot.xaxis.set_major_locator(mdates.WeekdayLocator()) 
    #aplot.xaxis.set_minor_locator(mdates.DayLocator())
    aplot.yaxis.set_minor_locator(AutoMinorLocator(4))
    plt_scatter1 = plt.plot(data.index, data.totale_positivi,label="Totale positivi", color="blue")
    plt_scatter2 = plt.plot(data.index, data.totale_ospedalizzati, label="Ospedalizzati", color="green")
    plt.ylim(0,data.totale_positivi.max().item() * 1.1)
    draw_reflines(plt, True)    
    
    annot_max(data.index,data.totale_positivi)    
    
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    
    plt.legend(by_label.values(), by_label.keys())


draw_scatters()



## Analisi predittiva variazione nuovi positivi

In [ ]:
from fbprophet import Prophet

delta_positivi = data[['variazione_totale_positivi']][20:]
delta_positivi['data'] = data[['variazione_totale_positivi']][20:].index
delta_positivi = delta_positivi.rename(columns={'data':'ds', 'variazione_totale_positivi':'y'})
# remove outliers
delta_positivi = delta_positivi.drop(delta_positivi[delta_positivi.y <= -2500].index)
delta_positivi = delta_positivi.drop(delta_positivi[delta_positivi.y >= 4500].index)
# model fit
model = Prophet(daily_seasonality=False, yearly_seasonality=False)
model.fit(delta_positivi)
future_days = model.make_future_dataframe(periods=60, freq='D')
# forecasting
forecast = model.predict(future_days)
fig = model.plot(forecast, ylabel='Numero nuovi positivi', xlabel='Giorni', figsize=(1200/72,720/72))
model.plot_components(forecast, figsize=(1200/72,720/72))
axes = fig.gca()
axes.grid(which='major', color='#999999', linestyle='--')
axes.grid(which='minor', color='#CCCCCC', linestyle=':')
axes.xaxis.set_major_locator(MultipleLocator(30))
axes.xaxis.set_minor_locator(AutoMinorLocator(10))
axes.yaxis.set_major_formatter(tick)
#axes.yaxis.set_major_locator(MultipleLocator(1000))
axes.yaxis.set_minor_locator(AutoMinorLocator(5))


In [ ]:
original, smoothed = covid.prepare_cases(data.totale_casi)
posteriors, log_likelihood = covid.get_posteriors(smoothed, sigma=.05)


In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
import pandas as pd

hdis = covid.highest_density_interval(posteriors, p=.9)
most_likely = posteriors.idxmax().rename('ML')

# Look into why you shift -1
result = pd.concat([most_likely, hdis], axis=1)

def plot_rt(result, ax, state_name):
    
    ax.set_title(f"Piemonte")
    
    # Colors
    ABOVE = [1,0,0]
    MIDDLE = [1,1,1]
    BELOW = [0,0,0]
    cmap = ListedColormap(np.r_[
        np.linspace(BELOW,MIDDLE,25),
        np.linspace(MIDDLE,ABOVE,25)
    ])
    color_mapped = lambda y: np.clip(y, .5, 1.5)-.5
    
    index = data.index
    values = result['ML'].values
    
    # Plot dots and line
    ax.plot(index, values, c='k', zorder=1, alpha=.25)
    ax.scatter(index,
               values,
               s=40,
               lw=.5,
               c=cmap(color_mapped(values)),
               edgecolors='k', zorder=2)
    
    # Aesthetically, extrapolate credible interval by 1 day either side
    lowfn = covid.interp1d(date2num(index),
                     result['Low_90'].values,
                     bounds_error=False,
                     fill_value='extrapolate')
    
    highfn = covid.interp1d(date2num(index),
                      result['High_90'].values,
                      bounds_error=False,
                      fill_value='extrapolate')
    
    extended = pd.date_range(start=pd.Timestamp('2020-02-24'),
                             end=index[-1]+pd.Timedelta(days=1))
    
    ax.fill_between(extended,
                    lowfn(date2num(extended)),
                    highfn(date2num(extended)),
                    color='k',
                    alpha=.1,
                    lw=0,
                    zorder=3)

    ax.axhline(1.0, c='k', lw=1, label='$R_t=1.0$', alpha=.25);
    
    # Formatting
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.set_minor_locator(mdates.DayLocator())
    
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.1f}"))
    ax.yaxis.tick_right()
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.margins(0)
    ax.grid(which='major', axis='y', c='k', alpha=.1, zorder=-2)
    ax.margins(0)
    ax.set_ylim(0.0, 5.0)
    fig.set_facecolor('w')

    
fig, ax = plt.subplots(figsize=(1200/72,720/72))

plot_rt(result, ax, 'Piemonte')
ax.set_title(f'$R_t$ giornaliero')
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

display(Markdown(f"## Stima $R_t$=  {result['ML'].values[-1]} ({result['Low_90'].values[-1]} - {result['High_90'].values[-1]})"))


*fonte*: [Dati DPC](https://github.com/pcm-dpc/COVID-19/blob/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv)